In [1]:
import pickle
from collections import Counter
from tqdm.auto import tqdm

In [2]:
def get_all_words_pred(corpus, vocab_size, word_predicate, **kwargs):
    """
    Get most common vocab_size words from corpus, given word_predicate
    """
    all_words = []
    counter = Counter()
    
    pt_emb = kwargs['pt_emb']
    min_freq = kwargs['min_freq']
    
    if isinstance(corpus, str):
        corpus = pickle.load(open(corpus, 'rb'))
    if isinstance(pt_emb, str):
        pt_emb = pickle.load(open(pt_emb, 'rb'))

    for sent in tqdm(corpus, desc="Question words processing"):
        for word in sent:
            if word_predicate(word):
                counter[word] += 1
                all_words.append(word)

    all_words = [word for word in all_words if counter[word] >= min_freq]

    return Counter(dict(Counter(all_words).most_common()[:vocab_size]))

In [3]:
CS_WORDS = """
    call function return value assign invoke procedure module import parameter argument
    syntax error replace process fork thread child semantic distance edit parameters arguments
    flow folder open close descriptor language comprehension list set dictionary braces curly
    sketch encoding kernel security algorithm loop recursive recursion mutual linux unix system 
    expression regular compile compiler bug fix workaround issue github git dinosaurs
""" .split()

In [7]:
question_words = pickle.load(open('../../corpus/python-stackoverflow/question_words_clean.pickle', 'rb'))
pt_emb = pickle.load(open('../../embeddings/glove.6B.200d.txt.pickle', 'rb'))
c = get_all_words_pred(
    corpus=question_words,
    vocab_size=8000,
    word_predicate=lambda w : w in pt_emb,
    min_freq=100, pt_emb=pt_emb # kwargs
)

In [8]:
print("Not found:", set(CS_WORDS) - set(c.keys()))
# c.most_common()[-100:]

Not found: {'dinosaurs'}
